# <font color='blue'>Uni-Facef - Pyspark - Atividade</font>

Essa atividade tem como objetivo descobrir um pouco mais sobre o cliente e traçar um perfil do mesmo. Visto que temos as informações pessoais do cliemte em um arquivo chamado "cliente_fake.csv", e os dados transacionais desse cliente no arquivo chamado "venda_fake". E idéia do exercicio é a manipulação dos dados de forma a fazer agregações, join nos dados, podendo ser necessário o uso de UDFs, gerando um Dataframe Final:

#### O Dataframe final deve estar agrupado por cliente (cliente_id), e conter as seguintes informações:

- cliente_id - O identificador do cliente
- idade - Calcular a idade do cliente (valor inteiro)
- recencia - Calcular a quantos meses faz que o cliente não compra (valor inteiro)
- qt_total_compras - Quantidade total de compras (pedidos distintos)
- vr_total_compras = Valor total de compras do cliente
- qt_total_itens - Quantidade total de itens comprados pelo cliente (produto_id distintos)
- qt_max_parcelas - Quantidade máxima de parcelas que o cliente já utilizou
- perfil_cliente - Deve mostrar o perfil de compra desse cliente, se compra somente "loja" ou somente "online", ou em ambos "multicanal";

OBS: O dataset "venda_fake" está a nível de itens. Pode haver mais de um item por pedido, ito é, o numero do pedido pode estar duplicado nodataset. 


In [1]:
from google.colab import files
uploaded = files.upload()

Saving cliente_fake.csv to cliente_fake.csv
Saving venda_fake.csv to venda_fake.csv


In [2]:
!pip install pyspark

     |████████████████████████████████| 204.2MB 61kB/s 
     |████████████████████████████████| 204kB 45.8MB/s 
  Created wheel for pyspark: filename=pyspark-3.0.1-py2.py3-none-any.whl size=204612243 sha256=956b7a3bf6a671c76d60e2ed37038fd845c15e36ef334a6de424cb7344c65037
  Stored in directory: /root/.cache/pip/wheels/5e/bd/07/031766ca628adec8435bb40f0bd83bb676ce65ff4007f8e73f
Successfully built pyspark


In [3]:
# coding: utf-8
from pyspark.sql import SparkSession

In [4]:
spark = SparkSession.builder \
    .appName('ConhecendoCliente') \
    .getOrCreate()

#### Dataset de cliente

In [5]:
df_cliente = spark.read \
    .option("delimiter", "|") \
    .csv('cliente_fake.csv', header=True)

df_cliente.show()

+--------------+----------+-------------+
|          nome|cliente_id|dt_nascimento|
+--------------+----------+-------------+
|Cliente 100013|    100013|   1964-06-21|
|Cliente 100014|    100014|   1964-05-24|
|Cliente 100015|    100015|   1946-10-18|
|Cliente 100016|    100016|   1972-03-24|
|Cliente 100017|    100017|   1982-01-19|
|Cliente 100018|    100018|   1983-04-26|
|Cliente 100019|    100019|   1971-09-09|
|Cliente 100020|    100020|   1989-06-18|
|Cliente 100007|    100007|   1976-11-04|
|Cliente 100008|    100008|   1980-12-01|
|Cliente 100009|    100009|   1981-03-05|
|Cliente 100010|    100010|   1984-03-20|
|Cliente 100011|    100011|   1977-11-22|
|Cliente 100012|    100012|   1974-02-04|
|Cliente 100001|    100001|   2009-04-16|
|Cliente 100002|    100002|   1974-08-29|
|Cliente 100003|    100003|   1981-12-05|
|Cliente 100004|    100004|   1968-08-08|
|Cliente 100005|    100005|   1982-11-29|
|Cliente 100006|    100006|   1948-04-01|
+--------------+----------+-------

#### Dataset de pedido

In [6]:
df_venda = spark.read \
    .option("delimiter", "|") \
    .csv('venda_fake.csv', header=True)

df_venda.show()

+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|     pedido_id|cliente_id|           dt_pedido|produto_id|qt_parcelas|   forma_pagto|vr_unit_item|qt_item|tp_canal|
+--------------+----------+--------------------+----------+-----------+--------------+------------+-------+--------+
|10000720200217|    100007|2020-02-17T00:00:...|   6253644|         20|cartao credito|         478|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   5313408|         20|cartao credito|      1357.2|      1|    loja|
|10000720200217|    100007|2020-02-17T00:00:...|   6253140|         20|cartao credito|         372|      1|    loja|
|10000720200625|    100007|2020-06-25T00:00:...|   6858780|          7|cartao credito|       317.8|      1|    site|
|10000120200626|    100001|2020-06-26T00:00:...|   7560984|         16|cartao credito|     1399.04|      1|    loja|
|10000920200614|    100009|2020-06-14T00:00:...|   7571280|     

In [7]:
import pyspark.sql.functions as sf

# DataFrame que calcula a idade do cliente (valor inteiro)
df_cliente_idade = df_cliente \
  .select(
      sf.col('cliente_id'),
      sf.col('dt_nascimento')) \
  .withColumn('idade', (sf.year(sf.current_date()) - sf.year('dt_nascimento')))

df_cliente_idade = df_cliente_idade \
    .select(
      sf.col('cliente_id'),
      sf.col('idade'))
    
df_cliente_idade.show()

+----------+-----+
|cliente_id|idade|
+----------+-----+
|    100013|   56|
|    100014|   56|
|    100015|   74|
|    100016|   48|
|    100017|   38|
|    100018|   37|
|    100019|   49|
|    100020|   31|
|    100007|   44|
|    100008|   40|
|    100009|   39|
|    100010|   36|
|    100011|   43|
|    100012|   46|
|    100001|   11|
|    100002|   46|
|    100003|   39|
|    100004|   52|
|    100005|   38|
|    100006|   72|
+----------+-----+



In [8]:
import pyspark.sql.functions as sf

# DataFrame que calcula a quantos meses faz que o cliente não compra (valor inteiro)
df_recencia = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('dt_pedido').cast('timestamp')) \
  .withColumn('recencia', 
              sf.months_between(sf.current_date(), 'dt_pedido') \
              .cast('integer')) \
  .groupBy('cliente_id') \
  .min('recencia') \
  .withColumnRenamed('min(recencia)', 'recencia')

df_recencia.show()


+----------+--------+
|cliente_id|recencia|
+----------+--------+
|    100010|       9|
|    100014|       4|
|    100008|       3|
|    100006|       2|
|    100005|       8|
|    100007|       4|
|    100020|       3|
|    100009|       4|
|    100004|       9|
|    100015|       2|
|    100016|       8|
|    100003|       2|
|    100019|       5|
|    100011|       8|
|    100012|       7|
|    100013|       2|
|    100017|      10|
|    100002|       8|
|    100018|      10|
|    100001|       4|
+----------+--------+



In [9]:
import pyspark.sql.functions as sf

# DataFrame que calcula a quantidade total de compras (pedidos distintos)
df_qtd_total_compras = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('pedido_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .withColumnRenamed('count', 'qt_total_compras')

df_qtd_total_compras.show()

+----------+----------------+
|cliente_id|qt_total_compras|
+----------+----------------+
|    100010|               1|
|    100014|               2|
|    100008|               1|
|    100006|               1|
|    100005|               2|
|    100007|               2|
|    100020|               1|
|    100009|               3|
|    100004|               2|
|    100015|               2|
|    100016|               1|
|    100003|               2|
|    100019|               3|
|    100011|               1|
|    100012|               1|
|    100013|               4|
|    100017|               1|
|    100002|               1|
|    100018|               1|
|    100001|               1|
+----------+----------------+



In [10]:
import pyspark.sql.functions as sf

# DataFrame que calcula o valor total de compras do cliente
df_valor_total_compras = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('vr_unit_item').cast('numeric(15,2)'),
      sf.col('qt_item').cast('integer')) \
  .withColumn('vr_total_items', 
              sf.col('vr_unit_item').cast('numeric(15,2)') * \
              sf.col('qt_item').cast('integer')) \
  .groupBy('cliente_id') \
  .sum('vr_total_items') \
  .withColumnRenamed('sum(vr_total_items)', 'vr_total_compras')
  
df_valor_total_compras.show()

+----------+----------------+
|cliente_id|vr_total_compras|
+----------+----------------+
|    100010|          898.97|
|    100014|         1746.01|
|    100008|         1399.86|
|    100006|         3170.72|
|    100005|         3207.28|
|    100007|         2525.00|
|    100020|          755.66|
|    100009|         4478.00|
|    100004|         6860.08|
|    100015|         2592.99|
|    100016|          181.08|
|    100003|         1904.95|
|    100019|         6114.96|
|    100011|         1324.80|
|    100012|         1099.98|
|    100013|         7053.58|
|    100017|         4176.88|
|    100002|         1280.07|
|    100018|         1437.96|
|    100001|         1399.04|
+----------+----------------+



In [11]:
import pyspark.sql.functions as sf

# DataFrame que calcula a quantidade total de itens comprados pelo cliente (produto_id distintos)
df_qtd_total_itens = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('produto_id')) \
  .distinct() \
  .groupBy('cliente_id') \
  .count() \
  .withColumnRenamed('count', 'qt_total_itens')
  
df_qtd_total_itens.show()

+----------+--------------+
|cliente_id|qt_total_itens|
+----------+--------------+
|    100010|             1|
|    100014|             3|
|    100008|             1|
|    100006|             3|
|    100005|             2|
|    100007|             4|
|    100020|             2|
|    100009|             4|
|    100004|             2|
|    100015|             3|
|    100016|             2|
|    100003|             3|
|    100019|             3|
|    100011|             2|
|    100012|             1|
|    100013|             7|
|    100017|             2|
|    100002|             1|
|    100018|             3|
|    100001|             1|
+----------+--------------+



In [12]:
import pyspark.sql.functions as sf

# DataFrame que calcula a quantidade máxima de parcelas que o cliente já utilizou
df_qtd_max_parcelas = df_venda \
  .select(
      sf.col('cliente_id'),
      sf.col('qt_parcelas').cast('integer')) \
  .distinct() \
  .groupBy('cliente_id') \
  .max('qt_parcelas') \
  .withColumnRenamed('max(qt_parcelas)', 'qt_max_parcelas')
  
df_qtd_max_parcelas.show()

+----------+---------------+
|cliente_id|qt_max_parcelas|
+----------+---------------+
|    100010|              9|
|    100014|             11|
|    100008|             14|
|    100006|              9|
|    100005|             15|
|    100007|             20|
|    100020|             11|
|    100009|             20|
|    100004|             24|
|    100015|              9|
|    100016|              9|
|    100003|             10|
|    100019|             13|
|    100011|              9|
|    100012|             14|
|    100013|             15|
|    100017|             17|
|    100018|             13|
|    100002|              9|
|    100001|             16|
+----------+---------------+



In [13]:
import pyspark.sql.functions as sf

# DataFrame que mostrar o perfil de compra desse cliente, se compra somente "loja" ou somente "online", ou em ambos "multicanal";

df_venda.createOrReplaceTempView("v_vendas")

df_perfil_cliente = spark.sql(
    """
    select distinct ve.cliente_id, 
      case when vi.qt_canal = 2 
               then 'multicanal' 
               else tp_canal 
               end perfil_cliente
      from (select cliente_id, count(distinct tp_canal) qt_canal
              from v_vendas
             group by cliente_id) vi
      join v_vendas ve on ve.cliente_id = vi.cliente_id
    """)

df_perfil_cliente.show()

+----------+--------------+
|cliente_id|perfil_cliente|
+----------+--------------+
|    100009|          loja|
|    100007|    multicanal|
|    100001|          loja|
|    100013|    multicanal|
|    100005|          loja|
|    100012|          loja|
|    100010|          loja|
|    100002|          loja|
|    100020|          loja|
|    100019|          loja|
|    100008|          loja|
|    100017|          loja|
|    100006|          loja|
|    100004|    multicanal|
|    100016|          loja|
|    100015|          loja|
|    100014|          loja|
|    100003|          loja|
|    100018|          loja|
|    100011|          loja|
+----------+--------------+



In [14]:
# DataFrame Final
df_final = df_cliente_idade \
   .join(df_recencia, 'cliente_id', 'inner') \
   .join(df_qtd_total_compras, 'cliente_id', 'inner') \
   .join(df_valor_total_compras, 'cliente_id', 'inner') \
   .join(df_qtd_total_itens, 'cliente_id', 'inner') \
   .join(df_qtd_max_parcelas, 'cliente_id', 'inner') \
   .join(df_perfil_cliente, 'cliente_id', 'inner') \

df_final.show()

+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|cliente_id|idade|recencia|qt_total_compras|vr_total_compras|qt_total_itens|qt_max_parcelas|perfil_cliente|
+----------+-----+--------+----------------+----------------+--------------+---------------+--------------+
|    100010|   36|       9|               1|          898.97|             1|              9|          loja|
|    100014|   56|       4|               2|         1746.01|             3|             11|          loja|
|    100008|   40|       3|               1|         1399.86|             1|             14|          loja|
|    100006|   72|       2|               1|         3170.72|             3|              9|          loja|
|    100005|   38|       8|               2|         3207.28|             2|             15|          loja|
|    100007|   44|       4|               2|         2525.00|             4|             20|    multicanal|
|    100020|   31|       3| 